In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from datetime import date
from scipy.optimize import minimize
from scipy.optimize import shgo
from dateutil.relativedelta import relativedelta
from datetime import timedelta
import json

In [2]:
#基金真实总股票持仓数据

chicang = pd.read_csv('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/output20210812cczb.csv',
                      header=0,encoding='gbk',engine='python')
chicang['end_dt'] =  chicang['end_dt'].apply(lambda x: x.replace('[','').replace(']','')) 
chicang['end_dt'] = pd.to_datetime(chicang['end_dt'])
chicang.mup_fund_ast_total_val_rati=chicang.mup_fund_ast_total_val_rati.apply(pd.to_numeric, errors='coerce')
chicang.mup_fund_ast_total_val_rati=chicang.mup_fund_ast_total_val_rati/100
l=chicang.loc[(chicang.mup_fund_ast_total_val_rati<0.6)].secu_abbr.to_list()
l=list(set(l))
chicang=chicang[~chicang.secu_abbr.isin(l)]

#增加基金类型标签
ftype=pd.read_csv('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/fund_type.csv',header=0,
                       engine='python')
flexible=ftype.loc[ftype.cstmz_insm_style=='灵活配置型']['secu_cd'].to_list()

#基金各股票持仓数据
output = pd.read_csv('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/output20210803v2.csv',
                      header=0,encoding='gbk',engine='python')

#剔除灵活配置型干扰
output=output[~output.secu_abbr.isin(flexible)]  
output=output[~output.secu_abbr.isin(l)]  
output.end_dt = output.end_dt.apply(lambda x: x.replace('[','').replace(']','')) 
output.end_dt=pd.to_datetime(output.end_dt)
output.hld_mval=output.hld_mval.apply(pd.to_numeric, errors='coerce')
output.ast_amt=output.ast_amt.astype(np.float64)
output=output.sort_values(by =['secu_cd','end_dt','hld_mval'])
output['ratio']=output.hld_mval/output.ast_amt
weight=output.pivot_table(index=['secu_cd','end_dt'],columns='insm_secu_cd',
                   values='ratio')
weight=weight.reset_index().set_index('end_dt')

#股票数据
stock_return= pd.read_csv('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/output20210803rate.csv',
                      header=0,engine='python')
stock_return.trans_dt = stock_return.trans_dt.apply(lambda x: x.replace('[','').replace(']','')) 
stock_return.trans_dt=pd.to_datetime(stock_return.trans_dt)
stock_return=stock_return.sort_values(by =['insm_secu_cd','trans_dt'])
rfr_all=stock_return.loc[:,['insm_secu_cd','trans_dt','day_rfr']]
rfr_all=rfr_all.pivot_table(index='trans_dt',columns='insm_secu_cd',
               values='day_rfr')

#中债财富指数
df_bond=pd.read_excel('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/债券指数.xlsx')
df_bond["day_rfr"] = df_bond["财富指数值"].pct_change(periods=1)*100
df_bond = df_bond.iloc[1:, :]
df_bond.drop(['财富指数值'],axis=1, inplace=True)
df_bond['日期'] = pd.to_datetime(df_bond['日期'])
bond_return=df_bond.pivot_table('day_rfr',index='日期',columns=['指数名称'])
bond_return.sort_index(inplace=True)

#现金利率
df_cash=pd.read_excel('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/现金收益率.xlsx')
df_cash['利率变动日期'] = pd.to_datetime(df_cash['利率变动日期'])
cash =df_cash.pivot_table('利率值',index='利率变动日期',columns=['指数名称'])
cash.sort_index(inplace=True)

df=pd.merge(bond_return,cash,how='outer', left_index=True, right_index=True)
df=df['2006-08-19':]
df['现金收益率']=df['现金收益率'].fillna(method='ffill')
df=df.dropna(axis=0,how='any')
df['现金收益率']=df['现金收益率']/365

#基金日度增长率
df_output = pd.read_csv('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/output.csv',
                        header=None,names=["编号", "基金名", "日期", "收益率","类型"],skiprows=1,
                        encoding='gbk',
                        engine='python')
df_output['日期'] =  df_output['日期'].apply(lambda x: x.replace('[','').replace(']','')) 
df_output['日期'] = pd.to_datetime(df_output['日期'])
df_output=df_output[~df_output.基金名.isin(flexible)]
df_output=df_output[~df_output.基金名.isin(l)]
df_output.收益率=df_output.收益率.apply(pd.to_numeric, errors='coerce')

fund_all=df_output.drop(['基金名'],axis=1)
fund_all.drop(['类型'],axis=1, inplace=True)
fund_all=fund_all.pivot_table('收益率',index='日期',columns=['编号'])
fund_all.sort_index(inplace=True)


fund_mix=df_output[df_output.类型=='混合型']
fund_stock=df_output[df_output.类型=='股票型']

#fund_mix.收益率=fund_mix.收益率.apply(pd.to_numeric, errors='coerce')
#fund_stock.收益率=fund_stock.收益率.apply(pd.to_numeric, errors='coerce')

fund_mix.drop(['基金名'],axis=1, inplace=True)
fund_mix.drop(['类型'],axis=1, inplace=True)
fund_mix=fund_mix.pivot_table('收益率',index='日期',columns=['编号'])
fund_mix.sort_index(inplace=True)

fund_stock.drop(['基金名'],axis=1, inplace=True)
fund_stock.drop(['类型'],axis=1, inplace=True)
fund_stock=fund_stock.pivot_table('收益率',index='日期',columns=['编号'])
fund_stock.sort_index(inplace=True)

#股票指数和基金日度增长率进行merge
merge_fund_mix=pd.merge(df,fund_mix, how='inner', left_index=True, right_index=True)
merge_fund_stock=pd.merge(df,fund_stock, how='inner', left_index=True, right_index=True)
merge_fund_all=pd.merge(df,fund_all, how='inner', left_index=True, right_index=True)

# vix数据
with open('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/vix.json') as json_data:
   obj = json.load(json_data)
   vix = pd.DataFrame(obj['respbody']['list'][0]['datas'], columns=['vix', 'date','ma20'])
vix.date=pd.to_datetime(vix.date,format='%Y%m%d', errors='ignore')
vix['std']=vix.vix.rolling(60).std(ddof=1)
v=vix.dropna(how='any',axis=0)
v=v.set_index(v['date'])

# 波动率数据
vol=pd.read_excel('/Users/alison/Desktop/thististest/FundEquityPositionEstimation/data/origin_data/万得全A指数.xlsx')

/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


2021-06-30 很多基金半年报持仓数据缺失，只显示部分持仓股？？
例如以下这只基金，别的时期稍微看了几眼没发现这个问题

In [6]:
t=output.loc[(output.secu_cd=='001313')&(output.end_dt=='2021-06-30')]
print("所有持仓之和",np.sum(t.ratio))

所有持仓之和 0.6159871444673183


In [7]:
#时间加权最小二乘:混合型基金
def Minimize_mix(A1,b1,x0):
    '''
    A1为变量x的矩阵（）
    b1为基金日度收益率
    x0为初始系数值
    '''    
    num_x = np.shape(A1)[1]
    num_day= np.shape(A1)[0]
    w=np.arange(1,num_day+1)
    w=np.exp(w/num_day)
    def my_func(x):
        ls = w*(b1 - np.dot(A1, x)) ** 2
        result = np.sum(ls)
        return result
    def g2(x):
        return 1 - np.sum(x)  # sum of stock and bond <= 1

    def g3(x):
        return 0.95 - np.sum(x[0:-1])  # sum of stock <= 0.95
    def g4(x):
        return np.sum(x[0:-1])-0.6  # sum of X >=0.6

    cons = ( {'type': 'ineq', 'fun': g2}
            , {'type': 'ineq', 'fun': g3}
            , {'type': 'ineq', 'fun': g4})
    bnds = [(0, 1)]
    for i in range(num_x - 1):
        bnds.append((0, 1))
    res=minimize(my_func,x0,method='SLSQP',bounds=bnds,constraints=cons)
    return res

In [8]:
#时间加权最小二乘:股票型基金
def Minimize_stock(A1,b1,x0):
    '''
    A1为变量x的矩阵（重仓股日收益、非重仓股日收益、债券日收益）
    b1为（基金日度收益率-现金收益率）
    x0为初始系数值
    '''            
    num_x = np.shape(A1)[1]
    num_day= np.shape(A1)[0]
    w=np.arange(1,num_day+1)
    w=np.exp(w/num_day)
    def my_func(x):
        ls = w*(b1 - np.dot(A1, x)) ** 2
        result = np.sum(ls)
        return result
    def g2(x):
        return 1 - np.sum(x)  # sum of stock and bond <= 1

    def g3(x):
        return 0.95 - np.sum(x[0:-1])  # sum of stock <= 0.95
    def g4(x):
        return np.sum(x[0:-1])-0.8  # sum of X >=0.8

    cons = ( {'type': 'ineq', 'fun': g2}
            , {'type': 'ineq', 'fun': g3}
            , {'type': 'ineq', 'fun': g4})
    bnds = [(0, 1)]
    for i in range(num_x - 1):
        bnds.append((0, 1))
    res=minimize(my_func,x0,method='SLSQP',bounds=bnds,constraints=cons)
    return res

In [17]:
def annual_stock(f,weight,stock_return,end_dt):
    """
    年报、半年报披露后用
    weight:pivot table,横轴为股票编号，纵轴为日期
    stock_return:df,窗口期内的股票日增长
    end_dt:半年报/年报的日期
    return df，row name：重仓股加权日增长，非重仓股加权日增长，column：窗口期内时间
    """
    
    weight=weight.dropna(how="any",axis=1)

    #将weight转置并从小到大排序
    w1=weight.T.sort_values(by=weight.T.columns.to_list())
    w1['cum_ratio']=np.cumsum(np.array(w1))
                
    #非重仓股list
    s_stock=w1.loc[w1['cum_ratio']<0.5].index.to_list()
    s_w=w1.loc[w1['cum_ratio']<0.5][end_dt]

    #重仓股list
    l_stock=w1.loc[w1['cum_ratio']>=0.5].index.to_list()
    l_w=w1.loc[w1['cum_ratio']>=0.5][end_dt]

    if (set(s_stock)<set(stock_return.columns.to_list()))&(set(l_stock)<set(stock_return.columns.to_list())):

        #从r60中提取出重仓非重仓股日增长
        s_matrix=stock_return.loc[:,s_stock]
        l_matrix=stock_return.loc[:,l_stock]
                    
        ###缺失股票日增长则补0
        s_matrix=s_matrix.fillna(0)
        l_matrix=l_matrix.fillna(0)
                    
        #加在原df保证有对应日期值
        stock_return['s_w_day_rfr']=np.dot(s_matrix,s_w)*2
        stock_return['l_w_day_rfr']=np.dot(l_matrix,l_w)*2

        #合并日收益表
        r=pd.merge(stock_return['l_w_day_rfr'],stock_return['s_w_day_rfr'],how='inner', 
               left_index=True,right_index=True)

    else:
        print(f+' Error:stock return not found')
        
    return r

In [10]:
def seasonal_top10stock(f,weight,stock_return):
    """
    季报披露后用
    weight:pivot table,横轴为股票编号，纵轴为日期
    stock_return:df,窗口期内的股票日增长
    return stock_return，row name：所有股票，重仓股日增长。column：窗口期内时间
    """

    ms=True
    
    weight=weight.dropna(how="any",axis=1)
    
    #重仓股list
    l_stock=weight.columns.to_list()

    #季报公布的股票的总持仓占比
    m=np.sum(np.array(weight),axis=1)    
    
    if set(l_stock) < set(stock_return.columns.to_list()):
        
        matrix=stock_return.loc[:,l_stock]
        matrix=matrix.fillna(0)
        
    #计算重仓股加权日增长
        stock_return['l_w_day_rfr']=np.dot(matrix,weight.T)
        stock_return['l_w_day_rfr']=stock_return['l_w_day_rfr']/m[0]
        ms=False
        
    else:
        print(f+' Error:stock return not found')
        
    return stock_return,m,l_stock,ms

In [11]:
def seasonal_rest_stock(f,previous_weight,l_stock,stock_return,m):
    """
    季报披露后用
    weight:pivot table,横轴为股票编号，纵轴为日期
    stock_return:df,窗口期内的股票日增长
    return stock_return，row name：所有股票，重仓股日增长。column：窗口期内时间
    """
    ms=True
    
    previous_weight=previous_weight.dropna(how="any",axis=1)
    
    #避免重复算进重仓股

    w1=previous_weight.drop(l_stock,axis=1)
    w1=w1.dropna(how='any',axis=1)

    #将weight转置并从小到大排序
    w1=w1.T.sort_values(by=w1.T.columns.to_list())
    w1['cum_ratio']=np.cumsum(np.array(w1))
    #非重仓股list
    s_stock=w1.loc[w1['cum_ratio']<=1-m[0]].index.to_list()


    if (set(s_stock)<set(stock_return.columns.to_list())):
        
        ms=False
        
        #从r60中提取出非重仓股日增长
        matrix1=stock_return.loc[:,s_stock]
        stock_return['s_w_day_rfr']=np.dot(matrix1,w1.drop('cum_ratio',axis=1).loc[s_stock,:])
                                            
        #调整权重为各股占小仓股市值的比例
        stock_return['s_w_day_rfr']=stock_return['s_w_day_rfr']/(1-m[0])
    
    else:
        print(f+' Error:stock not found in stock_return')
        
    return stock_return,ms

In [18]:
def prepare_optimization(fund,weighted_return,bond_cash_fund_return,window):
    """
    作用：准备套用minimize方程前的X矩阵和基金日增长-现金增长
    weighted_return：df，合并后的重仓与非重仓变量的加权日增长
    """
    
    period_r=pd.merge(weighted_return,bond_cash_fund_return,how='inner', 
                         left_index=True,right_index=True)
 
    #period_r的最后一行为出季报日期
    all_return=period_r.tail(window)

    data_array=np.array(all_return)
    A=data_array[:,np.arange(0,4)]
    A=A.astype(np.float64)
    
    #检查fund_bond_cash_return中是否有日期缺失增长值  
    missing = np.isnan(A).any()
    if missing:
        print(f+" Error:exist missing values in fund_bond_cash_return")
    
    if not(fund in all_return.columns.to_list()):
        print(fund+" Error:fund not found in fund_bond_cash_return.columns")
        
    if missing or not(fund in all_return.columns.to_list()):
        missing=True
        return 0,0,missing
            
    #对每一只基金进行最小二乘优化估算仓位    
    b=np.array(all_return[fund])-A[:,-1]
    b=b.astype(np.float64)
    x_val,y_val,missing=A[:,:-1],b,missing
    
    return x_val,y_val,missing
    

In [13]:
#季报披露后case：training data
merge_fund_all=pd.merge(df,fund_all, how='inner', left_index=True, right_index=True).tail(60)
stock_return=rfr_all.loc[rfr_all.index.isin(merge_fund_all.index)]
w=weight.loc[['2020-12-31','2021-03-31'],:]

In [14]:
#年报/半年报披露后case：training data
_merge_fund_all=pd.merge(df,fund_all, how='inner', left_index=True, right_index=True)
_merge_fund_all=_merge_fund_all.loc[_merge_fund_all.index<='2020-08-02']
_merge_fund_all=_merge_fund_all.tail(60)
_stock_return=rfr_all.loc[rfr_all.index.isin(_merge_fund_all.index)]
_w=weight.loc[['2020-03-31','2020-06-30'],:]

In [20]:
window=60
initial_guess=(0.5,0.4,0.05)
today='2021-08-02'
_today='2020-08-02'

#调出窗口期内：表一:股票日增长，表二：【编号，基金名，日期，日增长，类型】，表三：现金、中债国债日增长
#fund_info：【编号，基金名，日期，日增长，类型】的dataframe
#weight:dataframe 近两期的报表持仓信息:所有股票持仓和index为【时间，基金编号】的pivot table
#stock_return:窗口期内所有股票日增长和index为时间的pivot table
#bond_cash_fund_return：顺序:【中债国债日增长，现金，所有基金】和index为时间的pivot table

def estimate(fund_info,stock_return,initial_guess,weight,bond_cash_fund_return,today):
    """
    作用：直接求出含估算值等信息的df
    fund_info：【编号，基金名，日期，日增长，类型】的dataframe
    weight:dataframe 近两期的报表持仓信息:所有股票持仓和index为【时间，基金编号】的pivot table
    stock_return:窗口期内所有股票日增长和index为时间的pivot table
    bond_cash_fund_return：顺序:【中债国债日增长，现金，所有基金】和index为时间的pivot table
    """
    #d=datetime.strptime(date, "%Y-%m-%d")
    
    dates=[]
    beta0=[]
    beta1=[]
    beta2=[]
    beta3=[]
    stock_e=[]
    fund_name=[]
    fun=[]
    
    fund_list=list(set(fund_info.编号))
    
    for f in fund_list:
        
        #判断这只基金类型
        type_f=fund_info.loc[(fund_info.编号==f)]['类型'].all()
     
        #判断是季报还是半年报/年报的情况
        #先sort by date index
        weight=weight.sort_values(by='end_dt')
        end_dt=weight.index[-1]
        
        #调出这只基金的持仓信息
        w1=weight.loc[weight.secu_cd==f]
        w1=w1.drop(['secu_cd'],axis=1)
        
        if not(w1.shape[0]==0):
            
            if (end_dt.month ==6) or (end_dt.month ==12):

                w_annual=pd.DataFrame(w1.loc[end_dt]).T

                if not w_annual.shape[0]==0:
                    #得到加权后的重仓股和非重仓股日增长dataframe
                    weighted_return=annual_stock(f,w_annual,stock_return,end_dt)

                    #得到x_val为所有变量矩阵，y_val为基金日增长减去现金收益，missing则存在基金或债券无日增长
                    x_val,y_val,missing=prepare_optimization(f,weighted_return,bond_cash_fund_return,window)

                    if not missing:

                        if type_f == '混合型':
                            res=Minimize_mix(x_val,y_val, np.asarray(initial_guess))
                        else:
                            res=Minimize_stock(x_val,y_val, np.asarray(initial_guess))

                    if res.success:

                        dates.append(today)
                        beta0.append(res.x[0])
                        beta1.append(res.x[1])
                        beta2.append(res.x[2])
                        stock_e.append(res.x[0]+res.x[1])
                        fund_name.append(f)
                        fun.append(res.fun)

                    else:
                        print(f+' Error:optimization failed')


            else:

                #查看是否有完整两期季报
                if w1.shape[0]>=2:


                    #找出季报的weight为new_weight，调整为股票日增长为列
                    new_weight=pd.DataFrame(w1.loc[end_dt]).T
                    #上一期的weight为last_weight
                    previous_weight=w1.tail(1)

                    #求出重仓股日增长return包括股票日增长的df，重仓占比m，重仓股list，ms检验nan
                    l_return,m,l_stock,ms=seasonal_top10stock(f,new_weight,stock_return)

                    if not ms:
                        s_return,ms=seasonal_rest_stock(f,previous_weight,l_stock,stock_return,m)

                        if not ms:

                            #得出重仓与非重仓日增长
                            weighted_return=pd.merge(l_return['l_w_day_rfr'],s_return['s_w_day_rfr'],
                                       how='inner', left_index=True,right_index=True)

                            #得到x_val为所有变量矩阵，y_val为基金日增长减去现金收益，missing则存在基金或债券无日增长
                            x_val,y_val,missing=prepare_optimization(f,weighted_return,bond_cash_fund_return,window)

                            if not missing:

                                if type_f == '混合型':
                                    res=Minimize_mix(x_val,y_val, np.asarray(initial_guess))
                                else:
                                    res=Minimize_stock(x_val,y_val, np.asarray(initial_guess))

                                if res.success:

                                    dates.append(today)
                                    beta0.append(res.x[0])
                                    beta1.append(res.x[1])
                                    beta2.append(res.x[2])
                                    stock_e.append(res.x[0]+res.x[1])
                                    fund_name.append(f)
                                    fun.append(res.fun)

                                else:
                                    print(f+' Error:optimization failed')

                else:
                    print(f+' Error:financial report missing')
        else:
            print(f+' Error:financial report missing')
        
    df = { '日期':dates, '基金名称': fund_name, '基金股票仓位估计值': stock_e,
          'beta0':beta0, 'beta1':beta1,'beta2':beta2, 'fun':fun} 

    result = pd.DataFrame(df)
    
    return result

In [21]:
result=estimate(df_output,_stock_return,initial_guess,_w,_merge_fund_all,_today)

/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

010993 Error:financial report missing
010697 Error:financial report missing


/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


010791 Error:financial report missing
009394 Error:financial report missing


/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

010124 Error:financial report missing
006522 Error:optimization failed


/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

260104 Error:optimization failed


/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
result

,日期,基金名称,基金股票仓位估计值,beta0,beta1,beta2,fun
0,2020-08-02,540002,0.950000,0.446279,0.503721,3.468594e-14,11.379813
1,2020-08-02,001541,0.822072,0.403492,0.418580,0.000000e+00,15.011631
2,2020-08-02,167702,0.926471,0.403397,0.523074,0.000000e+00,1.328942
3,2020-08-02,004997,0.820298,0.500975,0.319324,3.774811e-02,14.734985
4,2020-08-02,005502,0.950000,0.351663,0.598337,1.386177e-13,5.175935
...,...,...,...,...,...,...,...
954,2020-08-02,510560,0.950000,0.445856,0.504144,9.451349e-14,1.950915
955,2020-08-02,005206,0.676527,0.345633,0.330893,0.000000e+00,2.892515
956,2020-08-02,020010,0.842005,0.431337,0.410669,0.000000e+00,7.224789
957,2020-08-02,163808,0.891495,0.405045,0.486450,4.235831e-17,4.452015
